# 💾 Loading Saved Models (Production Models)

<div style="background-color: #e3f2fd; padding: 15px; border-radius: 5px; border-left: 5px solid #2196F3;">
<b>📓 Notebook Information</b><br>
<b>Level:</b> Intermediate<br>
<b>Estimated Time:</b> 15 minutes<br>
<b>Prerequisites:</b> 03_model_integration.ipynb<br>
<b>Dataset:</b> Wine (sklearn)
</div>

---

## 🎯 Learning Objectives

By the end of this notebook, you will be able to:
- ✅ Save trained models to disk (.pkl, .joblib)
- ✅ Load models using DBDataset with `model_path`
- ✅ Validate production models with DeepBridge
- ✅ Understand supported formats (.pkl, .joblib, .h5, .onnx)
- ✅ Apply to production validation scenarios

---

## 📚 Table of Contents

1. [Introduction](#intro)
2. [Setup](#setup)
3. [Train and Save Model](#train)
4. [Load with model_path](#load)
5. [Supported Formats](#formats)
6. [Production Use Case](#production)
7. [Best Practices](#practices)
8. [Conclusion](#conclusion)
9. [Next Steps](#next)

<a id="intro"></a>
## 1. 📖 Introduction

### The Scenario

In production environments, you often have:
- 🤖 **Trained models saved to disk** (.pkl, .joblib, etc.)
- 📦 **New data to validate** against these models
- 🔍 **Need to re-validate** periodically

### Why Load Saved Models?

1. **Production Validation**: Validate existing production models
2. **Re-training**: Compare new model vs current production model
3. **A/B Testing**: Test multiple model versions
4. **Audit**: Reproduce validation results from historical models
5. **CI/CD**: Automate model validation in pipelines

### DeepBridge Makes it Easy!

```python
# Instead of:
# 1. Load model manually
# 2. Make predictions
# 3. Create DBDataset with predictions

# Just do:
dataset = DBDataset(
    data=df,
    target_column='target',
    model_path='my_model.pkl'  # ← Automatic!
)
```

**Let's learn how!** 🚀

<a id="setup"></a>
## 2. 🛠️ Setup

In [ ]:
# Standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import joblib
import pickle
from pathlib import Path

# sklearn
from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# DeepBridge
from deepbridge import DBDataset, Experiment

# Visualization settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("Set2")
%matplotlib inline

# Create directory for saved models
models_dir = Path('/tmp/deepbridge_models')
models_dir.mkdir(exist_ok=True)

print("✅ Setup complete!")
print(f"📁 Models directory: {models_dir}")

<a id="train"></a>
## 3. 🤖 Train and Save Model

First, let's train a model and save it - simulating what you would do in production.

In [ ]:
# Load Wine dataset
wine = load_wine()
df = pd.DataFrame(wine.data, columns=wine.feature_names)
df['target'] = wine.target

print("🍷 Wine Classification Dataset")
print(f"   Shape: {df.shape}")
print(f"   Features: {len(wine.feature_names)}")
print(f"   Classes: {len(np.unique(wine.target))}")
print(f"\n   First rows:")
df.head()

### Train Model

In [ ]:
# Simple split for training
from sklearn.model_selection import train_test_split

X = df.drop('target', axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Train RandomForest
model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=42
)

model.fit(X_train, y_train)

# Evaluate
train_acc = model.score(X_train, y_train)
test_acc = model.score(X_test, y_test)

print("✅ Model trained!")
print(f"\n📊 Performance:")
print(f"   Train Accuracy: {train_acc:.3f}")
print(f"   Test Accuracy: {test_acc:.3f}")

### Save Model - Multiple Formats

Let's save the model in different formats:

In [ ]:
# 1. Save with joblib (recommended for sklearn)
model_path_joblib = models_dir / 'wine_model.joblib'
joblib.dump(model, model_path_joblib)
print(f"✅ Saved with joblib: {model_path_joblib}")
print(f"   Size: {model_path_joblib.stat().st_size / 1024:.1f} KB")

# 2. Save with pickle
model_path_pkl = models_dir / 'wine_model.pkl'
with open(model_path_pkl, 'wb') as f:
    pickle.dump(model, f)
print(f"\n✅ Saved with pickle: {model_path_pkl}")
print(f"   Size: {model_path_pkl.stat().st_size / 1024:.1f} KB")

print("\n💡 Both formats work! joblib is usually preferred for sklearn models.")

<div style="background-color: #d4edda; padding: 10px; border-radius: 5px; border-left: 5px solid #28a745;">
<b>✅ Best Practice:</b> Use <code>joblib</code> for sklearn models - it's optimized for numpy arrays and is more efficient!
</div>

<a id="load"></a>
## 4. 🔄 Load Model with DBDataset

Now comes the magic! Let's load the saved model using DBDataset.

### Method 1: Using model_path (Recommended)

In [ ]:
# Create DBDataset with model_path
dataset = DBDataset(
    data=df,
    target_column='target',
    model_path=str(model_path_joblib),  # ← Just provide the path!
    test_size=0.2,
    random_state=42,
    dataset_name='Wine Classification'
)

print("✅ DBDataset created with saved model!")
print(f"\n📊 Details:")
print(f"   Model loaded from: {model_path_joblib.name}")
print(f"   Train samples: {len(dataset.train_data)}")
print(f"   Test samples: {len(dataset.test_data)}")
print(f"   Features: {len(dataset.features)}")

### What happened?

DBDataset automatically:
1. ✅ **Loaded the model** from disk
2. ✅ **Made predictions** on train and test data
3. ✅ **Stored predictions** in the dataset
4. ✅ **Validated** everything is consistent

**All with one parameter!** 🎉

### Verify Predictions

In [ ]:
# Access predictions
if hasattr(dataset, 'test_predictions'):
    y_pred = dataset.test_predictions
    y_true = dataset.get_target_data('test')
    
    accuracy = accuracy_score(y_true, y_pred)
    
    print("📊 Predictions from loaded model:")
    print(f"   Test Accuracy: {accuracy:.3f}")
    print(f"\n   Classification Report:")
    print(classification_report(y_true, y_pred, target_names=wine.target_names))
else:
    print("💡 Predictions will be computed when needed")

### Access Probabilities

In [ ]:
# Get predicted probabilities
if hasattr(dataset, 'test_probabilities'):
    test_proba = dataset.test_probabilities
    
    print("📊 Predicted Probabilities (first 5 samples):")
    print(f"   Shape: {test_proba.shape}")
    print(f"\n   Probabilities:")
    
    proba_df = pd.DataFrame(
        test_proba[:5],
        columns=[f'Class {i}' for i in range(test_proba.shape[1])]
    )
    display(proba_df.style.format("{:.3f}").background_gradient(cmap='Blues'))

<a id="formats"></a>
## 5. 📦 Supported Formats

DeepBridge supports multiple model formats:

### Supported Formats

| Format | Extension | Best For | Notes |
|--------|-----------|----------|-------|
| **Joblib** | `.joblib` | sklearn models | ✅ Recommended for sklearn |
| **Pickle** | `.pkl` | Any Python object | ⚠️ Security concerns |
| **Keras/TF** | `.h5`, `.keras` | Deep Learning | Neural networks |
| **ONNX** | `.onnx` | Cross-platform | Universal format |
| **PyTorch** | `.pt`, `.pth` | Deep Learning | Requires PyTorch |

### Example with .pkl

In [ ]:
# Load with .pkl file
dataset_pkl = DBDataset(
    data=df,
    target_column='target',
    model_path=str(model_path_pkl),  # ← .pkl works too!
    test_size=0.2,
    random_state=42
)

print("✅ Model loaded from .pkl file!")
print(f"   Format: pickle")
print(f"   Status: Working perfectly!")

<div style="background-color: #fff3cd; padding: 10px; border-radius: 5px; border-left: 5px solid #ffc107;">
<b>⚠️ Security Warning:</b> Only load models from trusted sources! Pickle files can execute arbitrary code.
</div>

<a id="production"></a>
## 6. 🏭 Production Use Case: Validation Pipeline

### Scenario
You have a model in production and want to:
1. Validate it with new data
2. Run all DeepBridge tests
3. Generate validation report

**Let's do it!**

In [ ]:
# Step 1: Load production model
print("📦 Step 1: Loading production model...")
dataset = DBDataset(
    data=df,
    target_column='target',
    model_path=str(model_path_joblib),
    test_size=0.2,
    random_state=42,
    dataset_name='Wine Classifier - Production Validation'
)
print("✅ Model loaded!\n")

# Step 2: Create Experiment
print("🔬 Step 2: Creating experiment...")
exp = Experiment(
    dataset=dataset,
    experiment_type='multiclass_classification',
    experiment_name='Production Model Validation',
    random_state=42
)
print("✅ Experiment created!\n")

# Step 3: Run quick tests
print("🧪 Step 3: Running validation tests (quick)...")
print("   This may take a moment...\n")

results = exp.run_tests(config='quick')

print("\n✅ Validation complete!")

### View Results

In [ ]:
# Summary of validation
print("📊 PRODUCTION MODEL VALIDATION SUMMARY")
print("=" * 60)

if hasattr(results, 'summary'):
    summary = results.summary()
    for test_name, test_results in summary.items():
        print(f"\n📋 {test_name.upper()}:")
        print(f"   Status: {test_results.get('status', 'N/A')}")
        print(f"   Score: {test_results.get('score', 'N/A')}")
else:
    print("\n✅ All tests completed successfully!")
    print("   Model passed basic validation")

print("\n" + "=" * 60)
print("\n💡 Generate full HTML report with: exp.save_html('report.html')")

<div style="background-color: #d4edda; padding: 15px; border-radius: 5px; border-left: 5px solid #28a745;">
<b>🎯 Production Pipeline:</b><br>
1. Load model with <code>model_path</code><br>
2. Create Experiment<br>
3. Run tests (robustness, uncertainty, etc.)<br>
4. Generate HTML report<br>
5. Make decision: approve/reject for production<br>
<br>
<b>All automated with DeepBridge!</b> 🚀
</div>

<a id="practices"></a>
## 7. 📝 Best Practices

### 1. Model Naming Convention

```python
# Good naming:
model_name = f"wine_classifier_v1.2_20250104.joblib"
#             └─name─────────┘ └─version─┘ └─date─┘
```

### 2. Save Metadata with Model

In [ ]:
# Save model + metadata
import json
from datetime import datetime

metadata = {
    'model_type': 'RandomForestClassifier',
    'version': '1.0',
    'train_date': datetime.now().isoformat(),
    'train_accuracy': float(train_acc),
    'test_accuracy': float(test_acc),
    'n_samples': len(df),
    'n_features': len(dataset.features),
    'hyperparameters': model.get_params()
}

# Save metadata
metadata_path = models_dir / 'wine_model_metadata.json'
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)

print("✅ Metadata saved!")
print(f"\n📄 Metadata:")
print(json.dumps(metadata, indent=2)[:300] + "...")

### 3. Version Control for Models

In [ ]:
# Directory structure example
print("📁 Recommended Directory Structure:")
print("""
models/
├── production/
│   ├── wine_classifier_v1.0.joblib
│   └── wine_classifier_v1.0_metadata.json
├── staging/
│   ├── wine_classifier_v1.1.joblib
│   └── wine_classifier_v1.1_metadata.json
└── archive/
    └── wine_classifier_v0.9.joblib
""")

print("💡 Benefits:")
print("   - Easy rollback")
print("   - A/B testing")
print("   - Audit trail")
print("   - Version comparison")

### 4. Automated Validation Script

In [ ]:
# Example validation function
def validate_production_model(model_path, data, target_column, output_dir):
    """
    Complete production model validation.
    
    Parameters:
    -----------
    model_path : str
        Path to saved model
    data : pd.DataFrame
        Validation data
    target_column : str
        Target column name
    output_dir : str
        Directory for reports
    
    Returns:
    --------
    bool : True if validation passes
    """
    print(f"🔬 Validating model: {Path(model_path).name}")
    
    # Create dataset
    dataset = DBDataset(
        data=data,
        target_column=target_column,
        model_path=model_path,
        test_size=0.2,
        random_state=42
    )
    
    # Create experiment
    exp = Experiment(
        dataset=dataset,
        experiment_type='multiclass_classification'
    )
    
    # Run tests
    results = exp.run_tests(config='full')
    
    # Save report
    report_path = Path(output_dir) / f'validation_report_{datetime.now():%Y%m%d_%H%M%S}.html'
    if hasattr(results, 'save_html'):
        results.save_html(str(report_path))
        print(f"✅ Report saved: {report_path}")
    
    # Validation criteria
    # (customize based on your requirements)
    passes = True  # Implement your criteria
    
    return passes

print("✅ Validation function defined!")
print("\n💡 Use in CI/CD pipeline:")
print("   python validate.py --model model.joblib --data data.csv")

<a id="conclusion"></a>
## 8. 🎉 Conclusion

### What you learned

Congratulations! You mastered loading saved models with DeepBridge! 🎊

In this notebook, you learned:
- ✅ How to save models with joblib and pickle
- ✅ How to load models using `model_path` parameter
- ✅ Supported formats (.joblib, .pkl, .h5, .onnx)
- ✅ Production validation workflow
- ✅ Best practices for model management
- ✅ How to create automated validation pipelines

### Key Takeaways

1. 💾 **model_path is powerful** - One parameter loads and integrates everything
2. 🔧 **Multiple formats supported** - Use what works for your stack
3. 🏭 **Production ready** - Easy to integrate in pipelines
4. 📝 **Always save metadata** - Version, date, metrics, hyperparameters
5. 🔒 **Security matters** - Only load models from trusted sources

---

### Notebook Metrics

```
🍷 Dataset: Wine (178 samples, 13 features, 3 classes)
🤖 Model: RandomForestClassifier
💾 Formats: .joblib and .pkl demonstrated
🔬 Validation: Full pipeline executed
⏱️ Time: ~15 minutes
```

<a id="next"></a>
## 9. 🎯 Next Steps

### Recommended

📘 **Next Notebook:** `05_probabilidades_precomputadas.ipynb`
- Optimize with pre-computed probabilities
- Save time with large models

### Alternative

📘 **Jump to:** `../03_validation_tests/06_comparacao_modelos.ipynb`
- Compare multiple saved models
- Choose best model for production

### Challenge

💪 **Production Pipeline Challenge!**
1. Train 3 different models (RandomForest, GradientBoosting, LogisticRegression)
2. Save all with proper naming and metadata
3. Load and validate each one
4. Compare results
5. Select best for "production"

---

## 📚 Additional Resources

- 📖 [DBDataset Documentation](../../../planejamento_doc/1-CORE/01-DBDATASET.md)
- 💻 [Model Serialization Guide](https://scikit-learn.org/stable/model_persistence.html)
- 🔒 [Security Best Practices](https://docs.python.org/3/library/pickle.html#comparison-with-json)

---

<div style="background-color: #e3f2fd; padding: 15px; border-radius: 5px; border-left: 5px solid #2196F3;">
<b>💬 Feedback</b><br>
Had issues or suggestions? <a href="https://github.com/DeepBridge-Validation/DeepBridge/issues">Open an issue on GitHub!</a>
</div>

---

<div style="text-align: center; padding: 20px;">
<h2>🎊 Great job completing this notebook! 🎊</h2>
<p style="font-size: 18px;">Continue mastering DBDataset with: <code>05_probabilidades_precomputadas.ipynb</code></p>
</div>